In [ ]:
%pip install tensorflow
%pip install tensorflow-text
%pip install tensorflow-hub
%pip install pymongo
%pip install pandas

In [3]:
from pymongo import MongoClient
import pandas as pd

client = MongoClient()
db = client.test
tweets = db['tweet']
tweets_aggregated = [
    {
        '$lookup': {
            'from': 'politiker', 
            'localField': 'author_id', 
            'foreignField': '_id', 
            'as': 'politiker'
        }
    }, {
        '$project': {
            'author_id': 1, 
            'text': 1, 
            'politiker.name': 1, 
            'politiker.username': 1, 
            'politiker.haltung': 1
        }
    }, {
        '$set': {
            'politiker': {
                '$first': '$politiker'
            }
        }
    }, {
        '$set': {
            'author_name': '$politiker.name', 
            'author_username': '$politiker.username', 
            'haltung': '$politiker.haltung'
        }
    }, {
        '$unset': 'politiker'
    }, {
        '$addFields': {
            'target': {
                '$cond': {
                    'if': {
                        '$eq': [
                            '$haltung', 'rechts'
                        ]
                    }, 
                    'then': 1.0, 
                    'else': 0.0
                }
            }
        }
    }
]
df = pd.DataFrame(list(tweets.aggregate(tweets_aggregated)))
df

,_id,author_id,text,author_name,author_username,haltung,target
0,1583836485031452673,833398209053605888,"Viele Grüße von unserer Klausurtagung, bei der...",Alice Weidel,Alice_Weidel,rechts,1.0
1,1583760099129978880,833398209053605888,Unser Land zuerst. #NurDreiWorte,Alice Weidel,Alice_Weidel,rechts,1.0
2,1583759897833132033,833398209053605888,Während die Bürger den Gürtel enger schnallen ...,Alice Weidel,Alice_Weidel,rechts,1.0
3,1583329087875321857,833398209053605888,Debatte zum #AfD-Antrag zur Wiedereinsetzung d...,Alice Weidel,Alice_Weidel,rechts,1.0
4,1582372615263354880,833398209053605888,RT @AfD: Der #Blackout wurde verschoben - aber...,Alice Weidel,Alice_Weidel,rechts,1.0
...,...,...,...,...,...,...,...
30994,1216680334047219712,743724264,Okay. Jeder Schritt zum #Klimaschutz ist gut. ...,Martin Schirdewan,schirdewan,links,0.0
30995,1216627559242387456,743724264,Profitgier und Kapitalinteresse sind nicht mit...,Martin Schirdewan,schirdewan,links,0.0
30996,1216439333571452929,743724264,RT @BernieSanders: It means you’re going to lose.,Martin Schirdewan,schirdewan,links,0.0
30997,1215953599567663105,743724264,4 Mio. Menschen sind jedes Jahr in #Europa zum...,Martin Schirdewan,schirdewan,links,0.0


In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
# preprocessor_url = "https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3"
preprocessor_url = "https://tfhub.dev/tensorflow/bert_multi_cased_preprocess/3"
preprocessor = hub.KerasLayer(preprocessor_url, name='preprocessor')
# encoder_url = "https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4"
encoder_url = "https://tfhub.dev/tensorflow/bert_multi_cased_L-12_H-768_A-12/4"
encoder = hub.KerasLayer(encoder_url, trainable=True, name='BERT')

In [ ]:
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
net = encoder(preprocessor(text_input))['pooled_output']
net = tf.keras.layers.Dropout(0.1)(net)
net = tf.keras.layers.Dense(10, activation=None, name='classifier')(net)
model = tf.keras.Model(text_input, net)
#model.summary()
loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)
metrics = tf.metrics.BinaryAccuracy()
optimizer = tf.keras.optimizers.Adam()
model.compile(optimizer=optimizer, loss=loss, metrics=metrics)


In [ ]:
model.fit(df['text'], df['target'])